In [25]:
import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [26]:

from nltk.corpus import wordnet as wn


def getSynonyms(word1):
    synonymList1 = []
    for data1 in word1:
        wordnetSynset1 = wn.synsets(data1)
        tempList1=[]
        tempList1.append(data1)
        #print(wordnetSynset1)
        
        for synset1 in wordnetSynset1:
            synLemmas = synset1.lemma_names()
            for i in range(len(synLemmas)):
                word = synLemmas[i].replace('_',' ')
                if word not in tempList1:
                    tempList1.append(word)
        
        spl = False            
        if(data1 == "what"):
            spl = True
            wordnetSynset1=wn.synsets("define")
        elif(data1 == "explain"):
            spl = True
            wordnetSynset1=wn.synsets("describe")            
        elif(data1 == "difference"):
            spl = True
            wordnetSynset1=wn.synsets("distinguish")
        elif(data1 == "advantage"):
            spl = True
            tempList1.extend(["pros","merits"])
            wordnetSynset1=wn.synsets("benefits")
        elif(data1 == "disadvantage"):
            spl = True
            tempList1.append("cons")
            wordnetSynset1=wn.synsets("demerits")
            
        if(spl):    
            for synset1 in wordnetSynset1:
                synLemmas = synset1.lemma_names()
                for i in range(len(synLemmas)):
                    word = synLemmas[i].replace('_',' ')
                    if word not in tempList1:
                        tempList1.append(word)
        synonymList1.append(tempList1)
    return synonymList1

print(getSynonyms(["demerits"]))

def getSynonymsIndex(word1, word2):
    len1 = len(word1)
    len2 = len(word2)
    
    if(len1+len2 == 0):
        return 0.5
    
    counter = 0
    d2 = getSynonyms(word2)
    #print(d2)
    
    for i in range(len(word1)):          
        for k in range(len(d2)):
            if word1[i] in d2[k]:
                counter += 1
                d2.pop(k)
                break

    syn_index = counter*2 / (len1 + len2)
    return syn_index

#print (getSynonymsIndex(word1, word2))     


def getMatchingIndex(word1, word2):
    len1 = len(word1)
    len2 = len(word2)
    
    if(len1+len2 == 0):
        return 0.5
    
    counter = 0
    for i in word1:
        for j in word2:
            if(i in j):
                counter+=1
                word2.remove(j)
                break
    match_index = counter*2 / (len1 + len2)
    return match_index    

[['demerits', 'demerit', 'fault']]


In [0]:
import nltk
import re
from nltk.stem import WordNetLemmatizer 
from pprint import pprint


def preProcess(sentence):
    lemmatizer = WordNetLemmatizer()
    sentence = re.sub(r'[^A-Za-z\s]+', ' ', sentence)
    sentence = re.sub(r'\s{2,}', ' ', sentence)


    #lemmatize
    tokens = nltk.word_tokenize(sentence)  
    #print(tokens)
    sentence_tmp = ""
    for word,pos in nltk.pos_tag(tokens):
        if type(word) == str:
            word = word.lower()
            
        if not pos == "DT":
            #print(pos)
            sentence_tmp+=(lemmatizer.lemmatize(word)) + " "  
    #print(sentence_tmp)
    return sentence_tmp

def extract_features(sentences):
    lemmatizer = WordNetLemmatizer()
    pre_features = [[],[]]

    counter=-1
    
    #print(sentences)
    for sentence in sentences:
        counter+=1

        sentence = preProcess(sentence)

        nouns = [] #empty to array to hold all nouns
        possesives = []
        adverbs = []
        adjectives = []
        verbs = []
        digrms = []
        questionWords = []

        tokens = nltk.word_tokenize(sentence)
        
        for word,pos in nltk.pos_tag(tokens):
            if (pos == 'NN' or pos == 'NNP' or pos == 'NNS' or pos == 'NNPS'):
                nouns.append(word)
            elif (pos == 'WDT' or pos == 'WP' or pos == 'WRB'):
                questionWords.append(word)                 
            elif (pos == 'POS'):
                possesives.append(word)
            elif (pos == 'RB' or pos == 'RBR' or pos == 'RBS'):
                adverbs.append(word)
            elif (pos == 'JJ' or pos == 'JJR' or pos == 'JJS'):
                adjectives.append(word)                
            elif (pos == 'VB' or pos == 'VBD' or pos == 'VBG' or pos == 'VBN' or pos == 'VBP' or pos == 'VBZ'):
                verbs.append(word)   

        
        bigrm = nltk.bigrams(tokens)
        for i in bigrm:
            digrms.append(" ".join(i))
        #print(digrms)
        
        
        from nltk.corpus import stopwords  
        stop_words = stopwords.words('english')
        list_words = [word for word in tokens if word not in stop_words]

        pre_features[counter].append(nouns) #0
        pre_features[counter].append(possesives) #1
        pre_features[counter].append(adverbs) #2
        pre_features[counter].append(adjectives) #3
        pre_features[counter].append(verbs) #4
        pre_features[counter].append(digrms) #5
        pre_features[counter].append(questionWords) #6
        pre_features[counter].append(list(list_words)) #7



    #print(pre_features[0])
    #print(pre_features[1])

    features = []

    for x in [0,2,3,4,6,7]:
        features.append(getSynonymsIndex(pre_features[0][x], pre_features[1][x]))

    for x in [1,5]:
        features.append(getMatchingIndex(pre_features[0][x], pre_features[1][x]))

    return(features)

In [28]:
sentence1 = "Bring the wine for me slowly" 
sentence2 = "to bring wine calmly"
sentences = []
sentence1 = preProcess(sentence1)
sentence2 = preProcess(sentence2)

sentences.append(sentence1)
sentences.append(sentence2)
print(extract_features(sentences))

[0.6666666666666666, 0.0, 0.5, 1.0, 0.5, 0.6666666666666666, 0.5, 0.2857142857142857]


In [29]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
#!cp /content/gdrive/My\ Drive/Colab\ Notebooks/project10x/train.tsv /content/gdrive/My\ Drive/Colab\ Notebooks/project1/glue_data/QQP/train.tsv

In [30]:
import os
import pandas as pd
#path = os.path.join(os.getcwd(),"project1","glue_data","QQP")


path = '/content/gdrive/My Drive/Colab Notebooks/project1/glue_data/QQP'
       
read_train = pd.read_csv(os.path.join(path,"train.tsv"),sep="\t")
read_test = pd.read_csv(os.path.join(path,"test.tsv"),sep="\t")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
#!python3 /content/gdrive/My\ Drive/Colab\ Notebooks/project1/download_glue_repo/download_glue_data.py --data_dir /content/gdrive/My\ Drive/Colab\ Notebooks/project1/glue_data/ --tasks QQP


In [7]:
print(read_test.head())
read_train.head()

   id  ...                                          question2
0   0  ...  Do you think that if Donald Trump were elected...
1   1  ...  What are the top ten Consumer-to-Business E-co...
2   2  ...  Why do people ask Quora questions instead of j...
3   3  ...                           Is social trade geniune?
4   4  ...  If universe and space is expanding? Does that ...

[5 rows x 3 columns]


,id,qid1,qid2,question1,question2,is_duplicate,Unnamed: 6
0,133273,213221,213222.0,How is the life of a math student? Could you d...,Which level of prepration is enough for the ex...,0,NaN
1,402555,536040,536041.0,How do I control my horny emotions?,How do you control your horniness?,1,NaN
2,360472,364011,490273.0,What causes stool color to change to yellow?,What can cause stool to come out as little balls?,0,NaN
3,150662,155721,7256.0,What can one do after MBBS?,What do i do after my MBBS ?,1,NaN
4,183004,279958,279959.0,Where can I find a power outlet for my laptop ...,"Would a second airport in Sydney, Australia be...",0,NaN


In [0]:
df_trainx = pd.DataFrame(read_train.iloc[:,3:5])
df_trainy = pd.DataFrame(read_train.iloc[:,5])



In [32]:
print(df_trainx.shape)
print(df_trainy.shape)
#print(df_trainx.head())


(363194, 2)
(363194, 1)


In [9]:
import time
count=0
dfx = []
to_remove = []

start = time.time()
for i in range(df_trainx.shape[0]):
    try:
        sentences = list(df_trainx.iloc[i])
        feature = extract_features(sentences)
    except:
        print(i)
        to_remove.append(i)
        continue
    
    #print(feature)
    dfx.append(feature)
    count+=1
    if count%20000 == 0:
        end = time.time()
        diff = end-start
        progress = (count/363194)*100
        print("Done : {}%".format(progress))
        print("Estimated time left : {} min".format((100 - progress)/progress*diff/60))
        


Done : 5.50669889921089%
Estimated time left : 24.284752174142604 min
36732
37801
Done : 11.01339779842178%
Estimated time left : 22.325586780832612 min
Done : 16.52009669763267%
Estimated time left : 20.801433356588948 min
Done : 22.02679559684356%
Estimated time left : 19.334730380911232 min
97828
Done : 27.533494496054452%
Estimated time left : 17.945312401145056 min
Done : 33.04019339526534%
Estimated time left : 16.532739021907318 min
130155
131430
139005
Done : 38.54689229447623%
Estimated time left : 15.154494432256222 min
Done : 44.05359119368712%
Estimated time left : 13.798412530579917 min
Done : 49.560290092898015%
Estimated time left : 12.41098335898086 min
Done : 55.066988992108904%
Estimated time left : 11.041490737712262 min
Done : 60.57368789131979%
Estimated time left : 9.687847673717917 min
228383
Done : 66.08038679053068%
Estimated time left : 8.335097324394052 min
248378
Done : 71.58708568974157%
Estimated time left : 6.973508876398801 min
Done : 77.09378458895246%


In [0]:
dfx = pd.DataFrame(dfx, columns = ['nouns', 'adverbs', 'adjectives',
                                   'verbs','questionWords', 'words',
                                   'possesives', 'digrms'])        


In [11]:
print(dfx.head())
#print(dfy.head())

      nouns  adverbs  adjectives  ...     words  possesives    digrms
0  0.000000      0.5         0.0  ...  0.000000         0.5  0.000000
1  0.000000      0.5         0.5  ...  0.400000         0.5  0.181818
2  0.400000      0.5         0.0  ...  0.400000         0.5  0.125000
3  1.000000      0.5         0.5  ...  0.666667         0.5  0.181818
4  0.153846      0.5         0.0  ...  0.200000         0.5  0.000000

[5 rows x 8 columns]


In [0]:

df_trainy.drop(df_trainy.index[to_remove], inplace=True)


In [33]:


print(dfx.shape)
print(df_trainy.shape)

#print(dfy.head())

(363181, 9)
(363194, 1)


In [0]:
import pandas as pd
colab_dir = '/content/gdrive/My Drive/Colab Notebooks/'
dfy = df_trainy.astype('bool')
#dfx.to_csv(colab_dir + 'project10x/train_dfx.csv')
#dfy.to_csv(colab_dir + 'project10x/train_dfy.csv')

dfx = pd.read_csv(colab_dir + 'project10x/train_dfx.csv')
dfy = pd.read_csv(colab_dir + 'project10x/train_dfy.csv')

In [0]:
'''
#import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt




xnn = []
for row in dfx.digrms:
    if(row>0.5):
        xnn.append(1.0)
    else:
        xnn.append(0.0)
        
xn = pd.DataFrame(xnn,columns = ['questionWords'] )
print(xn.head())

plt.subplot(2,1,1)
x = xn
y = dfy.is_duplicate
plt.scatter(x, y)

plt.subplot(2,1,2)
x = dfx.nouns
y = dfy.is_duplicate
plt.scatter(x, y)

plt.show()
'''

In [35]:

dfx = dfx.iloc[:,1:]
dfy = dfy.iloc[:,1]
print(dfx.shape)
print(dfy.shape)
print(dfx[20000:20005])
print(dfy[20000:20005])

#trainx.iloc[20000:20005,:])
#print(df_trainy.iloc[20000:20005,:])

(363181, 8)
(363181,)
          nouns  adverbs  adjectives  ...     words  possesives    digrms
20000  1.000000      0.5    1.000000  ...  1.000000         0.5  0.615385
20001  0.285714      0.4    0.666667  ...  0.428571         0.5  0.142857
20002  0.800000      0.5    0.000000  ...  0.500000         0.5  0.153846
20003  0.333333      0.0    0.500000  ...  0.222222         0.5  0.000000
20004  0.571429      0.0    0.500000  ...  0.666667         0.5  0.153846

[5 rows x 8 columns]
20000    False
20001    False
20002    False
20003    False
20004    False
Name: is_duplicate, dtype: bool


In [36]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC





modelLR = LogisticRegression()
modelLR.fit(dfx,dfy)


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [38]:
q1 = "what is computer?"
q2 = "what are advantage of computer?"

def predictOutput(q1,q2,model):
    q=[q1,q2]
    #print("hku")
    test_x = [extract_features(q)]
    test_x = pd.DataFrame(test_x, columns = ['nouns', 'adverbs', 'adjectives',
                                   'verbs', 'words', 'possesives',
                                   'digrms', 'questionWords'])  
    
    print(model.predict_proba(test_x)[0][1])
    
    print(model.predict_proba(test_x)[0][1] >= 0.80)
    return (model.predict(test_x))

#RandomForestClassifier(n_estimators=num_trees, random_state=9)
print(predictOutput(q1,q2,modelLR))
'''
for name, model in models:
    print(name)
    print(predictOutput(q1,q2,model))
'''
print("done")

0.8365232914580376
True
[ True]
done


In [44]:
import pickle
filename = '/content/gdrive/My Drive/Colab Notebooks/project10x/finalized_model.sav'
pickle.dump(modelLR, open(filename, 'wb'))
 
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = predictOutput(q1,q2,loaded_model)
print(result)

0.8365232914580376
True
[ True]


In [0]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import numpy as np
num_trees = 50
test_size = 0.1
seed=9

models = []
models.append(('LR', LogisticRegression(random_state=9)))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier(random_state=9)))
models.append(('RF', RandomForestClassifier(n_estimators=num_trees, random_state=9)))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC(random_state=9)))

(trainDataGlobal, testDataGlobal, trainLabelsGlobal, testLabelsGlobal) = train_test_split(np.array(dfx),
                                                                                          np.array(dfy),
                                                                                          test_size=test_size,
                                                                                          random_state=seed)
scoring = "accuracy"
results = []
names = []                                                                                          
# 5-fold cross validation
for name, model in models:
    kfold = KFold(n_splits=5, random_state=7)
    print(kfold)
    cv_results = cross_val_score(model, trainDataGlobal,
                                 trainLabelsGlobal, cv=kfold,
                                 scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)                                                                                          


KFold(n_splits=5, random_state=7, shuffle=False)


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  Futur

LR: 0.772632 (0.001377)
KFold(n_splits=5, random_state=7, shuffle=False)
LDA: 0.772629 (0.001372)
KFold(n_splits=5, random_state=7, shuffle=False)
KNN: 0.731373 (0.003681)
KFold(n_splits=5, random_state=7, shuffle=False)
CART: 0.679183 (0.001318)
KFold(n_splits=5, random_state=7, shuffle=False)
RF: 0.742635 (0.001451)
KFold(n_splits=5, random_state=7, shuffle=False)
NB: 0.759617 (0.008185)
KFold(n_splits=5, random_state=7, shuffle=False)


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [0]:
import pickle
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))
 
# some time later...
 
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, Y_test)
print(result)


from sklearn.externals import joblib
filename = 'finalized_model.sav'
joblib.dump(model, filename)
 
# some time later...
 
# load the model from disk
loaded_model = joblib.load(filename)
result = loaded_model.score(X_test, Y_test)
print(result)

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt

fig = pyplot.figure()
fig.suptitle('ML algorithm comparison')
ax = fig.add_subplot(111)
pyplot.boxplot(results)
ax.set_xticklabels(names)
pyplot.show()